In [1]:
from pathlib import Path
from tqdm import tqdm
import re
import pandas as pd
from datetime import datetime, timedelta
import sqlite3
import QueryStockDataDB
import stockdata
import OpenDartReader

import re

import requests

In [2]:
api_key = '92c176817e681dcc4ad263eb3fa5182792b0b7a3'
dart = OpenDartReader(api_key)

In [8]:
# 삼성전자 1999년~2019년 모든 정기보고서(최종보고서) (82건)
dart.list('005930', start='2010-01-01', end='2021-11-01', kind='A', final=True)

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00126380,삼성전자,005930,Y,반기보고서 (2021.06),20210817001416,삼성전자,20210817,
1,00126380,삼성전자,005930,Y,분기보고서 (2021.03),20210517001185,삼성전자,20210517,
2,00126380,삼성전자,005930,Y,사업보고서 (2020.12),20210309000744,삼성전자,20210309,연
3,00126380,삼성전자,005930,Y,분기보고서 (2020.09),20201116001248,삼성전자,20201116,
4,00126380,삼성전자,005930,Y,반기보고서 (2020.06),20200814001766,삼성전자,20200814,
5,00126380,삼성전자,005930,Y,분기보고서 (2020.03),20200515001451,삼성전자,20200515,
6,00126380,삼성전자,005930,Y,사업보고서 (2019.12),20200330003851,삼성전자,20200330,연
7,00126380,삼성전자,005930,Y,분기보고서 (2019.09),20191114001273,삼성전자,20191114,
8,00126380,삼성전자,005930,Y,반기보고서 (2019.06),20190814002218,삼성전자,20190814,
9,00126380,삼성전자,005930,Y,분기보고서 (2019.03),20190515001605,삼성전자,20190515,


In [116]:
reprt_code = '20210309000744'
url = f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={reprt_code}'
r = requests.get(url)
if r.status_code == 200 :
    string = r.text
else :
    print('''it's fail''')

In [117]:
def get_regex_search_node2(key, scriipt_tag) :
    return re.findall(r'node2\[\'' + key + '\'\]\s*=\s*(".*?")', scriipt_tag)

In [118]:
text_list = get_regex_search_node2('text', string)
id_list = get_regex_search_node2('id', string)
rcpNo_list = get_regex_search_node2('rcpNo', string)
dcmNo_list = get_regex_search_node2('dcmNo', string)
eleId_list = get_regex_search_node2('eleId', string)
offset_list = get_regex_search_node2('offset', string)
length_list = get_regex_search_node2('length', string)
dtd_list = get_regex_search_node2('dtd', string)
tocNo_list = get_regex_search_node2('tocNo', string)


print(len(text_list))
print(len(id_list))
print(len(rcpNo_list))
print(len(dcmNo_list))
print(len(eleId_list))
print(len(offset_list))
print(len(length_list))
print(len(dtd_list))
print(len(tocNo_list))


20
20
20
20
20
20
20
20
20


In [119]:
dic = {'text' : text_list,
        'id' : id_list,
        'rcpNo' : rcpNo_list, 
        'dcmNo' : dcmNo_list, 
        'eleId' : eleId_list, 
        'offset' : offset_list,
        'length' : length_list,
        'dtd' : dtd_list,
        'tocNo' : tocNo_list,
}

df = pd.DataFrame(dic)

In [120]:
df = df.applymap(lambda x: x.strip('"'))
df

,text,id,rcpNo,dcmNo,eleId,offset,length,dtd,tocNo
0,1. 회사의 개요,4,20210309000744,7861003,4,13800,241107,dart3.xsd,4
1,2. 회사의 연혁,5,20210309000744,7861003,5,254911,18093,dart3.xsd,5
2,3. 자본금 변동사항,6,20210309000744,7861003,6,273008,3576,dart3.xsd,6
3,4. 주식의 총수 등,7,20210309000744,7861003,7,276588,62302,dart3.xsd,7
4,5. 의결권 현황,8,20210309000744,7861003,8,338894,7901,dart3.xsd,8
5,6. 배당에 관한 사항 등,9,20210309000744,7861003,9,346799,18765,dart3.xsd,9
6,7. 정관에 관한 사항,10,20210309000744,7861003,10,365568,2291,dart3.xsd,10
7,1. 요약재무정보,13,20210309000744,7861003,13,664835,22260,dart3.xsd,13
8,2. 연결재무제표,14,20210309000744,7861003,14,687340,115750,dart3.xsd,14
9,3. 연결재무제표 주석,15,20210309000744,7861003,15,803094,555908,dart3.xsd,15


In [121]:
con = df.text.str.contains('.*연결재무제표$')
display(df.loc[con])
dic = df.loc[con].to_dict('records')[0]
dic

,text,id,rcpNo,dcmNo,eleId,offset,length,dtd,tocNo
8,2. 연결재무제표,14,20210309000744,7861003,14,687340,115750,dart3.xsd,14


{'text': '2. 연결재무제표',
 'id': '14',
 'rcpNo': '20210309000744',
 'dcmNo': '7861003',
 'eleId': '14',
 'offset': '687340',
 'length': '115750',
 'dtd': 'dart3.xsd',
 'tocNo': '14'}

In [149]:
params={
    'rcpNo': dic['rcpNo'],
    'dcmNo': dic['dcmNo'],
    'eleId': dic['eleId'],
    'offset': dic['offset'],
    'length': dic['length'],
    'dtd': dic['dtd']
    }

url = f'https://dart.fss.or.kr/report/viewer.do?'
r = requests.get(url, params=params)
html = r.text

table = pd.read_html(html)
length = len(table)
for k in range(0, length):
    title_candi = table[k].iat[0,0]
    lst = re.findall('.*재무상태표|손익계산서|현금흐름표', title_candi)
    if lst :
        print(k, title_candi, sep=':')
        display(table[k+1])

0:연결 재무상태표


,Unnamed: 0,제 52 기,제 51 기,제 50 기
0,자산,NaN,NaN,NaN
1,유동자산,198215579,181385260,174697424
2,현금및현금성자산,29382578,26885999,30340505
3,단기금융상품,92441703,76252052,65893797
4,단기상각후원가금융자산,2757111,3914216,2703693
5,단기당기손익-공정가치금융자산,71451,1727436,2001948
6,매출채권,30965058,35131343,33867733
7,미수금,3604539,4179120,3080733
8,선급비용,2266100,2406220,4136167
9,재고자산,32043145,26766464,28984704


3:연결 손익계산서


,Unnamed: 0,제 52 기,제 51 기,제 50 기
0,수익(매출액),236806988.0,230400881.0,243771415.0
1,매출원가,144488296.0,147239549.0,132394411.0
2,매출총이익,92318692.0,83161332.0,111377004.0
3,판매비와관리비,56324816.0,55392823.0,52490335.0
4,영업이익,35993876.0,27768509.0,58886669.0
5,기타수익,1384068.0,1778666.0,1485037.0
6,기타비용,2488902.0,1414707.0,1142018.0
7,지분법이익,506530.0,412960.0,539845.0
8,금융수익,12267600.0,10161632.0,9999321.0
9,금융비용,11318055.0,8274871.0,8608896.0


6:연결 포괄손익계산서


,Unnamed: 0,제 52 기,제 51 기,제 50 기
0,당기순이익(손실),26407832,21738865,44344857
1,기타포괄손익,"(3,673,905)",3016194,"(12,242)"
2,후속적으로 당기손익으로 재분류되지 않는 포괄손익,1788764,"(50,765)","(656,647)"
3,기타포괄손익-공정가치금융자산평가손익,2502733,1146599,"(235,865)"
4,관계기업 및 공동기업의 기타포괄손익에 대한 지분,5591,"(16,896)","(10,631)"
5,순확정급여부채(자산) 재측정요소,"(719,560)","(1,180,468)","(410,151)"
6,후속적으로 당기손익으로 재분류되는 포괄손익,"(5,462,669)",3066959,644405
7,관계기업 및 공동기업의 기타포괄손익에 대한 지분,"(48,888)",48649,6688
8,해외사업장환산외환차이,"(5,380,375)",3016499,590638
9,현금흐름위험회피파생상품평가손익,"(33,406)",1811,47079


12:연결 현금흐름표


,Unnamed: 0,제 52 기,제 51 기,제 50 기
0,영업활동 현금흐름,65287009,45382915,67031863
1,영업에서 창출된 현금흐름,68148810,56635791,78025064
2,당기순이익,26407832,21738865,44344857
3,조정,41618554,37442682,43604573
4,영업활동으로 인한 자산부채의 변동,122424,"(2,545,756)","(9,924,366)"
5,이자의 수취,2220209,2306401,1788520
6,이자의 지급,"(555,321)","(579,979)","(548,272)"
7,배당금 수입,243666,241801,215992
8,법인세 납부액,"(4,770,355)","(13,221,099)","(12,449,441)"
9,투자활동 현금흐름,"(53,628,591)","(39,948,171)","(52,240,453)"
